### Importing the required libraries

In [6]:
import numpy as np
import pandas as pd
import math
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix

### Listing down the filenames/ csv files from the required location

In [7]:
import os
for dirname, _, filenames in os.walk('/Users/priyanuj/CampusX_ML_Course'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/priyanuj/CampusX_ML_Course/ridge_regression_from_scratch_ols.ipynb
/Users/priyanuj/CampusX_ML_Course/Multiple_Linear_Regression_0520.ipynb
/Users/priyanuj/CampusX_ML_Course/train_digit.csv
/Users/priyanuj/CampusX_ML_Course/Titanic_Survival_Analysis_10212023.ipynb
/Users/priyanuj/CampusX_ML_Course/ridge_regression_from_scratch_gradient_descent_methodology.ipynb
/Users/priyanuj/CampusX_ML_Course/Gradient_Descent_function_video_1_first_code.ipynb
/Users/priyanuj/CampusX_ML_Course/test.csv
/Users/priyanuj/CampusX_ML_Course/decision_tree_clf_from_scratch.ipynb
/Users/priyanuj/CampusX_ML_Course/PCA_102123.ipynb
/Users/priyanuj/CampusX_ML_Course/perceptron_trick_code_classification.ipynb
/Users/priyanuj/CampusX_ML_Course/kNNClassifier_from_scratch_1012.ipynb
/Users/priyanuj/CampusX_ML_Course/logistic_regression_from_scratch.ipynb
/Users/priyanuj/CampusX_ML_Course/train.csv
/Users/priyanuj/CampusX_ML_Course/test_digit.csv
/Users/priyanuj/CampusX_ML_Course/train_data.csv
/Users/priyanuj/

### Importing titanic train and test data from the location and then dropping the un-necessary columns

In [9]:
#Importing titanic data from the required location
input_ads_pre = pd.read_csv('/Users/priyanuj/CampusX_ML_Course/train_data.csv')
print(input_ads_pre.shape)
input_ads_pre.head()

(792, 17)


,Unnamed: 0,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Title_1,Title_2,Title_3,Title_4,Emb_1,Emb_2,Emb_3
0,0,1,0,1,0.2750,0.014151,0,0,1,0.1,1,0,0,0,0,0,1
1,1,2,1,0,0.4750,0.139136,1,0,0,0.1,1,0,0,0,1,0,0
2,2,3,1,0,0.3250,0.015469,0,0,1,0.0,0,0,0,1,0,0,1
3,3,4,1,0,0.4375,0.103644,1,0,0,0.1,1,0,0,0,0,0,1
4,4,5,0,1,0.4375,0.015713,0,0,1,0.0,1,0,0,0,0,0,1


In [10]:
input_ads_pre.drop(columns = ['Unnamed: 0', 'Title_1', 'Title_2', 'Title_3', 'Title_4'], axis = 1, inplace = True)
input_ads_pre.head()

,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Emb_1,Emb_2,Emb_3
0,1,0,1,0.2750,0.014151,0,0,1,0.1,0,0,1
1,2,1,0,0.4750,0.139136,1,0,0,0.1,1,0,0
2,3,1,0,0.3250,0.015469,0,0,1,0.0,0,0,1
3,4,1,0,0.4375,0.103644,1,0,0,0.1,0,0,1
4,5,0,1,0.4375,0.015713,0,0,1,0.0,0,0,1


### Basic EDA

In [11]:
pd.DataFrame(input_ads_pre.isnull().sum()).T

,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Emb_1,Emb_2,Emb_3
0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
input_ads_pre.describe()

,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Emb_1,Emb_2,Emb_3
count,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000
mean,396.500000,0.386364,0.647727,0.368244,0.064677,0.243687,0.208333,0.547980,0.088636,0.185606,0.092172,0.720960
std,228.774999,0.487223,0.477980,0.162994,0.100987,0.429577,0.406373,0.498007,0.154485,0.389034,0.289451,0.448811
min,1.000000,0.000000,0.000000,0.008375,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,198.750000,0.000000,0.000000,0.275000,0.015469,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,396.500000,0.000000,1.000000,0.350000,0.028302,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,594.250000,1.000000,1.000000,0.437500,0.061045,0.000000,0.000000,1.000000,0.100000,0.000000,0.000000,1.000000
max,792.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Describing the target variable

In [13]:
# Total survived vs not-survived split in the training data
input_ads_pre['Survived'].value_counts()

0    486
1    306
Name: Survived, dtype: int64

### Shuffling the data

In [15]:
from sklearn.utils import shuffle

input_ads = shuffle(input_ads_pre, random_state = 100)
print(input_ads.shape)
input_ads.reset_index(drop = True, inplace = True)
input_ads.head()

(792, 12)


,PassengerId,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Emb_1,Emb_2,Emb_3
0,371,1,1,0.3125,0.108215,1,0,0,0.1,1,0,0
1,556,0,1,0.7750,0.051822,1,0,0,0.0,0,0,1
2,624,0,1,0.2625,0.015330,0,0,1,0.0,0,0,1
3,98,1,1,0.2875,0.123667,1,0,0,0.1,1,0,0
4,144,0,1,0.2375,0.013175,0,0,1,0.0,0,1,0


### Data Manipulation of Train and Test

In [23]:
target = 'Survived'

X = input_ads[[cols for cols in list(input_ads.columns) if target not in cols]]
y = input_ads[target]

print(X.columns)



# Since test data is placed in the input foler separately, we will import the test data set
test_ads_pre = pd.read_csv('/Users/priyanuj/CampusX_ML_Course/test_data.csv')
test_ads_pre.drop(columns = ['Unnamed: 0', 'Title_1', 'Title_2', 'Title_3', 'Title_4'], axis = 1, inplace = True)
test_ads = shuffle(test_ads_pre, random_state = 100)
test_ads.reset_index(drop = True, inplace = True)

# Splitting test dataset into X and y datasets (for supervised learning)
X_test = test_ads[[cols for cols in list(test_ads.columns) if target not in cols]]
y_test = test_ads[target]

print('Train % of total data: ', 100*X.shape[0]/ (X.shape[0] + X_test.shape[0]))

# Manipulation of datasets for convenience and consistency

X_arr = np.array(X)
X_test_arr = np.array(X_test)

y_arr = np.array(y).reshape(X_arr.shape[0],1)
y_test_arr = np.array(y_test).reshape(X_test_arr.shape[0],1)

# Summary
print(X_arr.shape)
print(X_test.shape)
print(y_arr.shape)
print(y_test_arr.shape)

Index(['PassengerId', 'Sex', 'Age', 'Fare', 'Pclass_1', 'Pclass_2', 'Pclass_3',
       'Family_size', 'Emb_1', 'Emb_2', 'Emb_3'],
      dtype='object')
Train % of total data:  88.78923766816143
(792, 11)
(100, 11)
(792, 1)
(100, 1)


## Decision Tree Classifier From Scratch

### UDF to calculate gini index of each node

In [28]:
# k = unique number of classes defined in a later function
def gini_node(arr_, k):
    
    class_elem_total = 0
    for class_ in k: #Iterating through each class in the node
        class_elem = (np.sum((arr_==class_).astype(int)))/(len(arr_))  # arr_ == 0 will give true or false and hence sum will work for both classes
        class_elem = class_elem**2
        class_elem_total = class_elem_total + class_elem
    
    gini_node = 1 - class_elem_total
    return gini_node

### UDF for gini index of a split (weighted by leafs)

In [29]:
def gini_split(left_arr, right_arr, k):
    
    # Total observations in each node
    m_left = len(left_arr)
    m_right = len(right_arr)
    m_total_node = m_left + m_right
    
    
    # Calculating gini index at each node
    gini_left = gini_node(left_arr, k)
    
    gini_right = gini_node(right_arr, k)
    
    # calculation of gini for the split
    
    if m_left == 0:
        gini_split = ((m_right/m_total_node) * gini_right)
        
    elif m_right == 0:
        gini_split = ((m_left/m_total_node) * gini_left)
        
    elif (m_left>0) & (m_right>0):
        gini_split = ((m_left/m_total_node) * gini_left) + ((m_right/m_total_node) * gini_right)
    
    return gini_split

### UDF for finding best split for a feature, greedy exact search

In [50]:
def feature_split_algo(data, col_idx, min_samples_split = 2, min_samples_leaf = 2):
    
    assert len(data[:, col_idx]) >= min_samples_split, "Data Insufficient -  Either include more data or reduce min_samples_split hyper-parameter"
    
    k = np.unique(data[:, -1]) #unique of all classes going into function
    
    # To be used for thresholds
    unique_vals = np.unique(data[:, col_idx])
    
    
    if len(unique_vals) > 1: # because if all values are same, there is no point in splitting with that column
        
        gini_node_ = gini_node(data[:, -1], k)
        
        splits_gini_dict = {}
        thresholds_discarded = []
        
        for threshold in unique_vals: # For each threshold possible
            
            left_split = data[data[:, col_idx] <= threshold] # Left extension of the tree
            left_split_target = left_split[:,-1]
            
            right_split = data[data[:, col_idx] > threshold] # Left extension of the tree
            right_split_target = right_split[:,-1]
            
            if (len(left_split_target) > min_samples_leaf) and (len(right_split_target) > min_samples_leaf): #Condition on mininum samples for a split to be eligible
                
                gini_split_ = gini_split(left_arr = left_split_target, right_arr = right_split_target, k = k)
                
                splits_gini_dict.update({threshold :  gini_split_}) #weighted gini of right and left node for each split
                
            else:
                # Discarding the threshold if condition not met
                thresholds_discarded.append(threshold)
                
        # Condition to avoid empty dictionary (if no split is feasible)
        if len(splits_gini_dict) > 0:
            
            min_gini = min(splits_gini_dict.values())
            
            # it is the threshold or the splitting value
            split_val = [key for key in splits_gini_dict if splits_gini_dict[key] == min_gini]
            
            # here we are obtaining the value of the split for the column where the gini impurity is minimum
            # for the column, we got the value of the splitting condition
            split_col_map_dict = {col_idx : split_val[0]}
            # for the column we got the value of the gini as well
            best_score_col_map_dict = {col_idx : min_gini}
            
            return split_col_map_dict, best_score_col_map_dict
        
        else:
            split_col_map_dict = {col_idx : np.nan}
            
            best_score_col_map_dict = {col_idx : np.nan}
            
            return split_col_map_dict, best_score_col_map_dict
    
    else:
        
        split_col_map_dict = {col_idx : np.nan}
            
        best_score_col_map_dict = {col_idx : np.nan}
        
        # Returning dict of the best split and their best score with their col idx as key
        return split_col_map_dict, best_score_col_map_dict

### UDF for overall best split finding algorithm (Greedy Exact Search)

In [55]:
def overall_split(data, col_idx_eligible):
    
    scores_dict = {}
    split_val_dict = {}
    
    # For a subset of columns required
    for col_idx in col_idx_eligible:
        
        split_val_dict_temp, scores_dict_temp = feature_split_algo(data=data, col_idx = col_idx)
        
        print(scores_dict_temp)
        
        scores_dict.update(scores_dict_temp)
        split_val_dict.update(split_val_dict_temp)
        
        
    #----------------------------------------------------------------------------------------------------------
    best_score_overall = min(scores_dict.values()) # Extracting the mninimum gini score across all columns
    best_score_col_idx = [key for key in scores_dict if scores_dict[key] == best_score_overall]
    #Extracting the col idx with the best score
    
    best_col_idx_split_val = split_val_dict[best_score_col_idx[0]]
    
    # Returning the dictionary with column idx and list of best split gini score and the threshold/splitting cond
    return {best_score_col_idx[0]: [best_score_overall, best_col_idx_split_val]}

### UDF for splitting the current dataframe by the best split found out through the above algorithm

In [56]:
def split_array(arr, arr_y, col_idx, split_val):
    
    # Splitting for X dataframe
    x_left = arr[arr[:, col_idx] <= split_val]
    x_right = arr[arr[:, col_idx] > split_val]
    
    # Splitting for y dataframe
    y_left = arr_y[arr[:, col_idx] <= split_val]
    y_right = arr_y[arr[:, col_idx] > split_val]
    
    return x_left, x_right, col_idx, split_val, y_left, y_right

# Class for the Classification Tree

In [59]:
class cart:
    
    def __init__(self):
        print(" in init")
       
    # UDF for CART
    def grow_tree(self, x_data, y_data, node_dict, max_depth, feat_idx_list = [0,1,4,5,7,9], min_samples_split = 5,
                 min_samples_leaf=2,depth=0,input_ads_=input_ads):
        
        # To restricting going beyond max_depth
        if depth <= max_depth:
            
            print('#------------------------- DEPTH: ', depth, ' -------------------------#')
            
            #Calculating best split overall
            
            split_dict = overall_split(data = np.append(x_data, y_data, axis = -1)
                                       , col_idx_eligible = feat_idx_list)
            
            split_col_idx = list(split_dict.keys())[0]
            gini_ = list(split_dict.values())[0][0]
            split_col_val = gini_ = list(split_dict.values())[0][1]
            
            #--------------------------------------------------------------------------------------
            print('1. ----------------> Entering root_node of depth : ', depth)
            
            #Splitting on the best split point found
            
            x_left, x_right, col_idx, split_val, y_left, y_right = split_array(arr=x_data
                                                                               , arr_y = y_data
                                                                               , col_idx = split_col_idx
                                                                               , split_val = split_col_val)
            
            #defining dictionary for the node of the tree
            node_dict = {
                
                'col': input_ads_.columns[split_col_idx], 'col_idx': split_col_idx,
                'threshold': split_col_val, 'val' : np.mean(y_data), 'n_class_0': len(y_data[y_data]==0),
                'n_class_1': len(y_data[y_data==1]), 'n_vals': len(y_data)
            } #save the information
            
            print('2. ----------------> First : \n', node_dict)
            
            print('3. ----------------> Entering left of depth : ', depth)
            
            node_dict['left'] = self.grow_tree(x_data=x_left,
                                               y_data=y_left,
                                               feat_idx_list=feat_idx_list,
                                               node_dict={},
                                               max_depth=max_depth,
                                               min_samples_split=min_samples_split,
                                               min_samples_leaf=min_samples_leaf,
                                               depth=depth+1)   
            #-----------------------------------------------------------------------
            if node_dict['left']==None:
                print('4. -------> None:\n')
            
            # right hand side trees
            print('5. -------> Entering right of depth:',depth)
            node_dict['right'] = self.grow_tree(x_data=x_right,
                                               y_data=y_right,
                                               feat_idx_list=feat_idx_list,
                                               node_dict={},
                                               max_depth=max_depth,
                                               min_samples_split=min_samples_split,
                                               min_samples_leaf=min_samples_leaf,
                                               depth=depth+1)
            
            if node_dict['right']==None:
                print('6. --------> None:\n')
            
            #print('After :\n',node_dict)
            #Error Handling
            try:
                self.depth += 1   # increase the depth since we call fit once
            except:
                print('7. -------> Entering except---')
                return node_dict
            
        elif depth>max_depth:
            return None
        
        elif (len(y_data)<min_samples_split) | (len(y_data)<min_samples_leaf):
            return None
        
        elif node_dict is None:
            return None

        
        #Returns the fully expanded tree
        return node_dict

### Setting up the class object and then invoking it for the whole classification tree building

In [60]:
cart_ = cart()
tree_dict_ = cart_.grow_tree(x_data=X_arr,
                             y_data=y_arr,
                             node_dict={},
                             max_depth=2,
                             input_ads_=input_ads)

 in init
#------------------------- DEPTH:  0  -------------------------#
{0: 0.47171505085580834}
{1: 0.3310508721751884}
{4: 0.440020146625283}
{5: 0.46906867720264844}
{7: 0.4574129473382337}
{9: 0.47414944306555556}
1. ----------------> Entering root_node of depth :  0
2. ----------------> First : 
 {'col': 'Survived', 'col_idx': 1, 'threshold': 0.0, 'val': 0.38636363636363635, 'n_class_0': 792, 'n_class_1': 306, 'n_vals': 792}
3. ----------------> Entering left of depth :  0
#------------------------- DEPTH:  1  -------------------------#
{0: 0.37265443811933374}
{1: nan}
{4: 0.33847434922703745}
{5: 0.3543193271546412}
{7: 0.33123223334569263}
{9: 0.37575147104496653}
1. ----------------> Entering root_node of depth :  1
2. ----------------> First : 
 {'col': 'Pclass_3', 'col_idx': 7, 'threshold': 0.3, 'val': 0.7491039426523297, 'n_class_0': 279, 'n_class_1': 209, 'n_vals': 279}
3. ----------------> Entering left of depth :  1
#------------------------- DEPTH:  2  ---------------

### UDF for prediction of a single row in test data

In [61]:
tree_dict_

{'col': 'Survived',
 'col_idx': 1,
 'threshold': 0.0,
 'val': 0.38636363636363635,
 'n_class_0': 792,
 'n_class_1': 306,
 'n_vals': 792,
 'left': {'col': 'Pclass_3',
  'col_idx': 7,
  'threshold': 0.3,
  'val': 0.7491039426523297,
  'n_class_0': 279,
  'n_class_1': 209,
  'n_vals': 279,
  'left': {'col': 'Fare',
   'col_idx': 4,
   'threshold': 0.0,
   'val': 0.8,
   'n_class_0': 250,
   'n_class_1': 200,
   'n_vals': 250,
   'left': None,
   'right': None},
  'right': {'col': 'Fare',
   'col_idx': 4,
   'threshold': 0.0,
   'val': 0.3103448275862069,
   'n_class_0': 29,
   'n_class_1': 9,
   'n_vals': 29,
   'left': None,
   'right': None}},
 'right': {'col': 'Fare',
  'col_idx': 4,
  'threshold': 0.0,
  'val': 0.18908382066276802,
  'n_class_0': 513,
  'n_class_1': 97,
  'n_vals': 513,
  'left': {'col': 'Pclass_3',
   'col_idx': 7,
   'threshold': 0.1,
   'val': 0.1396508728179551,
   'n_class_0': 401,
   'n_class_1': 56,
   'n_vals': 401,
   'left': None,
   'right': None},
  'right

In [62]:
def single_row_pred(test_x_,max_depth,temp_tree_dict): #Takes in the tree dict from training

    for i in range(max_depth): #For all depth
        
        #print('------ depth :',i)
        threshold = temp_tree_dict['threshold']
        split_col_idx = temp_tree_dict['col_idx']

        tree_dict_left = temp_tree_dict['left']
        tree_dict_right = temp_tree_dict['right']
        
        #Traversing into left side
        if (test_x_[split_col_idx]<=threshold) & (tree_dict_left!=None) & (tree_dict_right!=None):

            temp_tree_dict = tree_dict_left

            if (temp_tree_dict['left']==None) & (temp_tree_dict['right']==None):
                prediction = temp_tree_dict['val']
                #pred_list.append(prediction)

        #Traversing into right side
        elif (test_x_[split_col_idx]>threshold) & (tree_dict_left!=None) & (tree_dict_right!=None):

            temp_tree_dict = tree_dict_right

            if (temp_tree_dict['left']==None) & (temp_tree_dict['right']==None):
                prediction = temp_tree_dict['val']
                #pred_list.append(prediction)

        #If end of the tree is reached, generate predictions 
        elif (tree_dict_left==None) & (tree_dict_right==None):

            prediction = temp_tree_dict['val']
            
            
    return prediction

### UDF for overall prediction

In [63]:
def predict_tree(test_data,max_depth,tree_object=tree_dict_,threshold=0.5): #Takes in tree dictionary and threshold of proabability
    
    pred_list = []
    
    #For each row in test data
    for idx in range(len(test_data)):
        
        #Sngle row prediction calculation
        pred = np.round(single_row_pred(test_x_=test_data[idx],
                                        max_depth=max_depth,
                                        temp_tree_dict=tree_object),3)
        pred_list.append(pred)
        
    print('Length of preds :',len(pred_list))
    
    #Converting into array
    pred_proba = np.array(pred_list)
    
    #Converting into class predictions (binary) based on threshold
    pred_list = (np.array(pred_proba)>threshold).astype(int)
        
    return pred_list

### Prediction from the Decision Tree built from scratch

In [64]:
preds_manual = predict_tree(test_data=X_test_arr
                            ,max_depth=2,
                            tree_object=tree_dict_,
                            threshold=0.5)

#----------------------------------------------------------------------------------------------------
print('Total predictions :', len(preds_manual))
print('Unique of predictions :',np.unique(preds_manual))
preds_manual[0:10]

#----------------------------------------------------------------------------------------------------
#Evaluating the model
score = roc_auc_score(y_test_arr, preds_manual)
print('1. ROC AUC: %.3f' % score)
print('2. Accuracy :',accuracy_score(y_test_arr, preds_manual))
print('3. Classification Report -\n',classification_report(y_test_arr, preds_manual))
print('4. Confusion Matrix - \n',confusion_matrix(y_test_arr, preds_manual))

Length of preds : 100
Total predictions : 100
Unique of predictions : [0 1]
1. ROC AUC: 0.779
2. Accuracy : 0.81
3. Classification Report -
               precision    recall  f1-score   support

           0       0.83      0.89      0.86        64
           1       0.77      0.67      0.72        36

    accuracy                           0.81       100
   macro avg       0.80      0.78      0.79       100
weighted avg       0.81      0.81      0.81       100

4. Confusion Matrix - 
 [[57  7]
 [12 24]]


### Sklearn Benchmarking

In [65]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(random_state=100,max_depth=2,min_samples_split=5,min_samples_leaf=2)
dt_clf.fit(X_arr[:,[0,1,4,5,7,9]],y_arr)

sklearn_preds = dt_clf.predict(X_test_arr[:,[0,1,4,5,7,9]])

#------------------------------------------------------------------------------------------------------
#Evaluating the model
score = roc_auc_score(y_test_arr, sklearn_preds)
print('1. ROC AUC: %.3f' % score)
print('2. Accuracy :',accuracy_score(y_test_arr, sklearn_preds))
print('3. Classification Report -\n',classification_report(y_test_arr, sklearn_preds))
print('4. Confusion Matrix - \n',confusion_matrix(y_test_arr, sklearn_preds))

1. ROC AUC: 0.779
2. Accuracy : 0.81
3. Classification Report -
               precision    recall  f1-score   support

           0       0.83      0.89      0.86        64
           1       0.77      0.67      0.72        36

    accuracy                           0.81       100
   macro avg       0.80      0.78      0.79       100
weighted avg       0.81      0.81      0.81       100

4. Confusion Matrix - 
 [[57  7]
 [12 24]]


### End!!!!!!!!!!!!!!!!!!!!!!!!